In [1]:
import pandas as pd

import json
from pandas import json_normalize 
from ast import literal_eval


In [2]:
# Load the users, receipts, and brands json data 
users = [json.loads(line) for line in open('users.json','r')]
receipts = [json.loads(line) for line in open('receipts.json','r')]
brands = [json.loads(line) for line in open('brands.json','r')]

In [3]:
# Normalizefrom json data to a table 
df_users = json_normalize(users)
df_receipts = json_normalize(receipts)
df_brands = json_normalize(brands)

In [4]:
# rename the dataset
df_users = df_users.rename(columns={'_id.$oid':'id' , 'createdDate.$date': 'CreatedDate',
                                    'lastLogin.$date' :'lastLogin'})
df_receipts = df_receipts.rename(columns={'_id.$oid':'id' , 'createDate.$date': 'CreatedDate',
                                    'dateScanned.$date' :'dateScanned','finishedDate.$date' :'finishedDate',
                                         'modifyDate.$date' :'modifyDate','pointsAwardedDate.$date' :'pointsAwardedDate',
                                         'purchaseDate.$date' :'purchaseDate'})
df_brands = df_brands.rename(columns={'_id.$oid':'id' , 'cpg.$id.$oid': 'CPG_id','cpg.$ref': 'CPG_ref'})


In [5]:
# convert each element of the column,'rewardsReceiptItemList', into a row.
df_receipts = df_receipts.explode('rewardsReceiptItemList')
df_receipts.reset_index(inplace=True)


In [6]:
df_receipts = df_receipts.fillna({'rewardsReceiptItemList':'{}'})


In [7]:
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(lambda x:str(x))

In [8]:
df_receipts['rewardsReceiptItemList'] = df_receipts['rewardsReceiptItemList'].apply(literal_eval)

In [9]:
#make a new dataset with the column of 'df_receipts'dataset', rewardsReceiptItemList 
df_ItemList = pd.json_normalize(df_receipts['rewardsReceiptItemList'],errors='ignore')

In [10]:
id= df_receipts.id
id = pd.DataFrame(id)

In [11]:
df_ItemList_final= pd.merge(id,df_ItemList, left_index=True, right_index=True, how='outer')

In [13]:
#Rename the column of the new dataset
df_receipts = df_receipts.rename(columns={'_id.$oid':'id' , 'createDate.$date': 'CreateDate','dateScanned.$date': 'dateScanned',
                                    'finishedDate.$date' :'finishedDate' , 'modifyDate.$date' :'modifyDate',
                                         'pointsAwardedDate.$date' :'pointsAwardedDate','purchaseDate.$date' :'purchaseDate'})

In [14]:
df_receipts =df_receipts.drop('rewardsReceiptItemList',axis =1)


In [15]:
# convert 13-digit unixtime to timestamp format
import datetime
df_receipts['CreatedDate'] = df_receipts['CreatedDate'].apply(lambda time : datetime.datetime.fromtimestamp(int(time)/1000))
df_receipts['dateScanned'] =df_receipts[~df_receipts['dateScanned'].isnull()].loc[:,'dateScanned'].apply(lambda time : 
                                                                    datetime.datetime.fromtimestamp(int(time)/1000))
df_receipts['finishedDate'] =df_receipts[~df_receipts['finishedDate'].isnull()].loc[:,'finishedDate'].apply(lambda time : 
                                                                    datetime.datetime.fromtimestamp(int(time)/1000))
df_receipts['modifyDate'] =df_receipts[~df_receipts['modifyDate'].isnull()].loc[:,'modifyDate'].apply(lambda time : 
                                                                    datetime.datetime.fromtimestamp(int(time)/1000))
df_receipts['pointsAwardedDate'] =df_receipts[~df_receipts['pointsAwardedDate'].isnull()].loc[:,'pointsAwardedDate'].apply(lambda time : 
                                                                    datetime.datetime.fromtimestamp(int(time)/1000))
df_receipts['purchaseDate'] =df_receipts[~df_receipts['purchaseDate'].isnull()].loc[:,'purchaseDate'].apply(lambda time : 
                                                                    datetime.datetime.fromtimestamp(int(time)/1000))



In [33]:

df_receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   index                    7381 non-null   int64         
 1   bonusPointsEarned        5980 non-null   float64       
 2   bonusPointsEarnedReason  5980 non-null   object        
 3   pointsEarned             6253 non-null   object        
 4   purchasedItemCount       6897 non-null   float64       
 5   rewardsReceiptStatus     7381 non-null   object        
 6   totalSpent               6946 non-null   object        
 7   userId                   7381 non-null   object        
 8   id                       7381 non-null   object        
 9   CreatedDate              7381 non-null   datetime64[ns]
 10  dateScanned              7381 non-null   datetime64[ns]
 11  finishedDate             5970 non-null   datetime64[ns]
 12  modifyDate               7381 non-

In [18]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   active        495 non-null    bool   
 1   role          495 non-null    object 
 2   signUpSource  447 non-null    object 
 3   state         439 non-null    object 
 4   id            495 non-null    object 
 5   CreatedDate   495 non-null    int64  
 6   lastLogin     433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [19]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   id            1167 non-null   object
 6   CPG_id        1167 non-null   object
 7   CPG_ref       1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [20]:
df_ItemList_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7381 entries, 0 to 7380
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  7381 non-null   object 
 1   barcode                             3090 non-null   object 
 2   description                         6560 non-null   object 
 3   finalPrice                          6767 non-null   object 
 4   itemPrice                           6767 non-null   object 
 5   needsFetchReview                    813 non-null    object 
 6   partnerItemId                       6941 non-null   object 
 7   preventTargetGapPoints              358 non-null    object 
 8   quantityPurchased                   6767 non-null   float64
 9   userFlaggedBarcode                  337 non-null    object 
 10  userFlaggedNewItem                  323 non-null    object 
 11  userFlaggedPrice                    299 non

In [21]:
# Export to csv file to do mySql work. 
df_users.to_csv('users.csv', index=False)
df_receipts.to_csv('receipts.csv', index=False)
df_brands.to_csv('brands.csv', index=False)
df_ItemList_final.to_csv('Receipts_itemlist.csv', index=False)

In [29]:
len(pd.unique(df_receipts['id']))

1119

In [30]:
len(pd.unique(df_users['id']))

212

In [32]:
len(pd.unique(df_ItemList_final['id']))

1119

Data Quality Check

In [34]:
# missing value
for a,b in df_users.isnull().mean().items():
    if b > 0:
        print(a, b)



signUpSource 0.09696969696969697
state 0.11313131313131314
lastLogin 0.12525252525252525


In [37]:
for a,b in df_receipts.isnull().mean().items():
    if b > 0:
        print(a, b)


bonusPointsEarned 0.1898116786343314
bonusPointsEarnedReason 0.1898116786343314
pointsEarned 0.15282482048502913
purchasedItemCount 0.06557377049180328
totalSpent 0.058935103644492615
finishedDate 0.1911665086031703
pointsAwardedDate 0.1762633789459423
purchaseDate 0.06205121257282211


In [38]:
for a,b in df_ItemList_final.isnull().mean().items():
    if b > 0:
        print(a, b)


barcode 0.5813575396287766
description 0.11123154044167458
finalPrice 0.08318656008670912
itemPrice 0.08318656008670912
needsFetchReview 0.8898523235333966
partnerItemId 0.05961251862891207
preventTargetGapPoints 0.951497087115567
quantityPurchased 0.08318656008670912
userFlaggedBarcode 0.9543422300501287
userFlaggedNewItem 0.9562389920065032
userFlaggedPrice 0.9594905839317166
userFlaggedQuantity 0.9594905839317166
needsFetchReviewReason 0.9703292236824279
pointsNotAwardedReason 0.9539357810594771
pointsPayerId 0.82834304294811
rewardsGroup 0.7654789323939846
rewardsProductPartnerId 0.6925890800704512
userFlaggedDescription 0.9722259856388024
originalMetaBriteBarcode 0.9903807072212437
originalMetaBriteDescription 0.998645170031161
brandCode 0.6477442081018833
competitorRewardsGroup 0.96274217585693
discountedItemPrice 0.2183985909768324
originalReceiptItemText 0.21961793794878742
itemNumber 0.9792711014767647
originalMetaBriteQuantityPurchased 0.9979677550467416
pointsEarned 0.874407

In [39]:
for a,b in df_brands.isnull().mean().items():
    if b > 0:
        print(a, b)


category 0.1328191945158526
categoryCode 0.5569837189374465
topBrand 0.5244215938303342
brandCode 0.20051413881748073


In [43]:
#duplicated data 

num_dupli_users =df_users.duplicated().sum()
num_dupli_receipts =df_receipts.duplicated().sum()
num_dupli_itemlist =df_ItemList_final.duplicated().sum()
num_dupli_brands =df_brands.duplicated().sum()
print('# of Duplicated data in users dataset:',num_dupli_users)
print('# of Duplicated data in receipts dataset:',num_dupli_receipts)
print('# of Duplicated data in itemlist dataset:',num_dupli_itemlist)
print('# of Duplicated data in brands dataset:',num_dupli_brands)

# of Duplicated data in users dataset: 283
# of Duplicated data in receipts dataset: 6262
# of Duplicated data in itemlist dataset: 0
# of Duplicated data in brands dataset: 0
